<div style="background-color:#9dedde; padding: 15px">
<h2>Projet 7 : Implémentez un modèle de Scoring</h2>
</div>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly import tools
import plotly.express as px
from datetime import datetime
import datetime

# pd.set_option('display.max_columns', 200)
# pd.set_option('display.max_rows', 1000)

# **Chargement des données**

In [ ]:
df = pd.read_csv("app_train_sample.csv")

In [ ]:
df

# **Analyse de la forme du dataset**

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df.dtypes.value_counts()

In [ ]:
df.select_dtypes(object).nunique().sort_values(ascending=False)

In [ ]:
(df.select_dtypes(object).nunique().sort_values(ascending=False) / len(df)).round(3)

# **NaN & Duplicated**

In [ ]:
df.duplicated().sum()

In [ ]:
# Calculer les taux de NaN pour chaque colonne
nan_counts = df.isna().sum() / len(df) * 100

# Créer un graphique à barres horizontales pour visualiser les taux de NaN
plt.figure(figsize=(10, 30))
nan_counts.sort_values().plot(kind="barh")
plt.title("Taux de NaN pour chaque colonne")
plt.xlabel("Pourcentage de NaN")
plt.ylabel("Colonnes")
plt.show()

In [ ]:
# Calculer les taux de NaN pour chaque colonne
nan_counts = df.isna().sum() / len(df) * 100

# Créer une dataframe triée par ordre décroissant des taux de NaN
sorted_nan_counts = nan_counts.sort_values(ascending=False).reset_index()
sorted_nan_counts.columns = ["Colonnes", "Taux_NaN"]


# Créer une coloration (cmap) pour la dataframe
cmap = plt.get_cmap("Reds")
sorted_nan_counts.style.background_gradient(cmap="Reds", subset=["Taux_NaN"])

In [ ]:
# Calculer les taux de NaN pour chaque colonne
nan_percentages = df.isna().mean() * 100

# Identifier les colonnes avec plus de 60% de NaN
cols_to_drop = nan_percentages[nan_percentages > 60].index

# Supprimer les colonnes identifiées
df = df.drop(cols_to_drop, axis=1)

In [ ]:
# Calculer les taux de NaN pour chaque colonne
nan_counts = df.isna().sum() / len(df) * 100

# Créer une dataframe triée par ordre décroissant des taux de NaN
sorted_nan_counts = nan_counts.sort_values(ascending=False).reset_index()
sorted_nan_counts.columns = ["Colonnes", "Taux_NaN"]


# Créer une coloration (cmap) pour la dataframe
cmap = plt.get_cmap("Greens")
sorted_nan_counts.style.background_gradient(cmap="Greens", subset=["Taux_NaN"])

In [ ]:
df.duplicated(subset="SK_ID_CURR").sum()

# **Data Analyse**

In [ ]:
df.describe()

In [ ]:
df.describe(include=object)

# **SELECTION DES COLONNES**

In [ ]:
df

In [ ]:
# Liste des noms de colonnes
columns = {
    # Informations générales sur les clients :
    "SK_ID_CURR": "ID_CLIENT",
    "TARGET": "CIBLE",
    "NAME_CONTRACT_TYPE": "TYPE_CONTRAT",
    # Informations personnelles sur le client :
    "CODE_GENDER": "GENRE",
    "FLAG_OWN_CAR": "PROPRIETAIRE_VOITURE",
    "FLAG_OWN_REALTY": "PROPRIETAIRE_BIEN_IMMO",
    "CNT_CHILDREN": "NBR_ENFANTS",
    "AMT_INCOME_TOTAL": "REVENU_TOTAL",
    "NAME_TYPE_SUITE": "NOM_TYPE_SUITE",
    "NAME_INCOME_TYPE": "TYPE_REVENUS",
    "NAME_EDUCATION_TYPE": "NIVEAU_EDUCATION",
    "NAME_FAMILY_STATUS": "STATUT_FAMILIAL",
    "NAME_HOUSING_TYPE": "TYPE_LOGEMENT",
    # Informations démographiques et régionales :
    "REGION_POPULATION_RELATIVE": "POPULATION_REGION",
    "REG_REGION_NOT_LIVE_REGION": "REG_NON_RESIDENCE_REGION",
    "REG_REGION_NOT_WORK_REGION": "REG_NON_REGION_TRAVAIL",
    "LIVE_REGION_NOT_WORK_REGION": "RESIDENCE_NON_TRAVAIL_REGION",
    "REG_CITY_NOT_LIVE_CITY": "REG_NON_RESIDENCE_VILLE",
    "REG_CITY_NOT_WORK_CITY": "REG_NON_TRAVAIL_VILLE",
    "LIVE_CITY_NOT_WORK_CITY": "RESIDENCE_NON_TRAVAIL_VILLE",
    # Informations temporelles :
    "DAYS_BIRTH": "JOURS_NAISSANCE",
    "DAYS_EMPLOYED": "JOURS_EMPLOI",
    "DAYS_REGISTRATION": "JOURS_ENREGISTREMENT",
    "DAYS_ID_PUBLISH": "JOURS_ID_PUBLICATION",
    "WEEKDAY_APPR_PROCESS_START": "JOUR_PROCESSUS_DEBUT",
    "HOUR_APPR_PROCESS_START": "HEURE_PROCESSUS_DEBUT",
    "DAYS_LAST_PHONE_CHANGE": "JOURS_DERNIER_CHANGEMENT_TELEPHONE",
    # Informations sur l'organisation du client :
    "ORGANIZATION_TYPE": "TYPE_ORGANISATION",
    # Informations provenant de sources externes :
    "EXT_SOURCE_1": "SOURCE_EXTERNE_1",
    "EXT_SOURCE_2": "SOURCE_EXTERNE_2",
    "EXT_SOURCE_3": "SOURCE_EXTERNE_3",
    # Informations sur les biens immobiliers du client :
    "AMT_CREDIT": "TOTAL_CREDIT",
    "AMT_ANNUITY": "REMB_MENSUEL",
    "AMT_GOODS_PRICE": "MONTANT_PRIX_BIENS",
    "APARTMENTS_AVG": "APPARTEMENTS_MOYENNE",
    "BASEMENTAREA_AVG": "SUPERFICIE_SOUTERRAINE_MOYENNE",
    "YEARS_BEGINEXPLUATATION_AVG": "DEBUT_EXPLOITATION_MOYENNE",
    "ELEVATORS_AVG": "ASCENSEURS_MOYENNE",
    "ENTRANCES_AVG": "ENTREES_MOYENNE",
    "FLOORSMAX_AVG": "ETAGES_MAX_MOYENNE",
    "LANDAREA_AVG": "SUPERFICIE_TERRAIN_MOYENNE",
    "LIVINGAREA_AVG": "SUPERFICIE_HABITABLE_MOYENNE",
    "NONLIVINGAREA_AVG": "SUPERFICIE_NON_HABITABLE_MOYENNE",
    "APARTMENTS_MODE": "APPARTEMENTS_MODE",
    "BASEMENTAREA_MODE": "SUPERFICIE_SOUTERRAINE_MODE",
    "YEARS_BEGINEXPLUATATION_MODE": "DEBUT_EXPLOITATION_MODE",
    "ELEVATORS_MODE": "ASCENSEURS_MODE",
    "ENTRANCES_MODE": "ENTREES_MODE",
    "FLOORSMAX_MODE": "ETAGES_MAX_MODE",
    "LANDAREA_MODE": "SUPERFICIE_TERRAIN_MODE",
    "LIVINGAREA_MODE": "SUPERFICIE_HABITABLE_MODE",
    "NONLIVINGAREA_MODE": "SUPERFICIE_NON_HABITABLE_MODE",
    "APARTMENTS_MEDI": "APPARTEMENTS_MEDIANE",
    "BASEMENTAREA_MEDI": "SUPERFICIE_SOUTERRAINE_MEDIANE",
    "YEARS_BEGINEXPLUATATION_MEDI": "DEBUT_EXPLOITATION_MEDIANE",
    "ELEVATORS_MEDI": "ASCENSEURS_MEDIANE",
    "ENTRANCES_MEDI": "ENTREES_MEDIANE",
    "FLOORSMAX_MEDI": "ETAGES_MAX_MEDIANE",
    "LANDAREA_MEDI": "SUPERFICIE_TERRAIN_MEDIANE",
    "LIVINGAREA_MEDI": "SUPERFICIE_HABITABLE_MEDIANE",
    "NONLIVINGAREA_MEDI": "SUPERFICIE_NON_HABITABLE_MEDIANE",
    "HOUSETYPE_MODE": "MODELE_TYPE_LOGEMENT",
    "TOTALAREA_MODE": "SUPERFICIE_TOTALE_MODE",
    "WALLSMATERIAL_MODE": "MODELE_MATERIAUX_MURS",
    "EMERGENCYSTATE_MODE": "MODE_URGENCE",
    # Informations sociales du client :
    "OBS_30_CNT_SOCIAL_CIRCLE": "OBS_30_CNT_CERCLE_SOCIAL",
    "DEF_30_CNT_SOCIAL_CIRCLE": "DEF_30_CNT_CERCLE_SOCIAL",
    "OBS_60_CNT_SOCIAL_CIRCLE": "OBS_60_CNT_CERCLE_SOCIAL",
    "DEF_60_CNT_SOCIAL_CIRCLE": "DEF_60_CNT_CERCLE_SOCIAL",
    # Informations sur les documents du client :
    "FLAG_MOBIL": "FLAG_MOBILE",
    "FLAG_EMP_PHONE": "FLAG_TELEPHONE_EMPLOYE",
    "FLAG_WORK_PHONE": "FLAG_TELEPHONE_TRAVAIL",
    "FLAG_CONT_MOBILE": "FLAG_TELEPHONE_MOBILE",
    "FLAG_PHONE": "FLAG_TELEPHONE",
    "FLAG_EMAIL": "FLAG_EMAIL",
    "FLAG_DOCUMENT_2": "FLAG_DOCUMENT_2",
    "FLAG_DOCUMENT_3": "FLAG_DOCUMENT_3",
    "FLAG_DOCUMENT_4": "FLAG_DOCUMENT_4",
    "FLAG_DOCUMENT_5": "FLAG_DOCUMENT_5",
    "FLAG_DOCUMENT_6": "FLAG_DOCUMENT_6",
    "FLAG_DOCUMENT_7": "FLAG_DOCUMENT_7",
    "FLAG_DOCUMENT_8": "FLAG_DOCUMENT_8",
    "FLAG_DOCUMENT_9": "FLAG_DOCUMENT_9",
    "FLAG_DOCUMENT_10": "FLAG_DOCUMENT_10",
    "FLAG_DOCUMENT_11": "FLAG_DOCUMENT_11",
    "FLAG_DOCUMENT_12": "FLAG_DOCUMENT_12",
    "FLAG_DOCUMENT_13": "FLAG_DOCUMENT_13",
    "FLAG_DOCUMENT_14": "FLAG_DOCUMENT_14",
    "FLAG_DOCUMENT_15": "FLAG_DOCUMENT_15",
    "FLAG_DOCUMENT_16": "FLAG_DOCUMENT_16",
    "FLAG_DOCUMENT_17": "FLAG_DOCUMENT_17",
    "FLAG_DOCUMENT_18": "FLAG_DOCUMENT_18",
    "FLAG_DOCUMENT_19": "FLAG_DOCUMENT_19",
    "FLAG_DOCUMENT_20": "FLAG_DOCUMENT_20",
    "FLAG_DOCUMENT_21": "FLAG_DOCUMENT_21",
    # Informations sur les demandes de crédit au bureau de crédit du client :
    "AMT_REQ_CREDIT_BUREAU_HOUR": "NBR_DEMANDES_CREDIT_BUREAU_HEURE",
    "AMT_REQ_CREDIT_BUREAU_DAY": "NBR_DEMANDES_CREDIT_BUREAU_JOUR",
    "AMT_REQ_CREDIT_BUREAU_WEEK": "NBR_DEMANDES_CREDIT_BUREAU_SEMAINE",
    "AMT_REQ_CREDIT_BUREAU_MON": "NBR_DEMANDES_CREDIT_BUREAU_MOIS",
    "AMT_REQ_CREDIT_BUREAU_QRT": "NBR_DEMANDES_CREDIT_BUREAU_TRIMESTRE",
    "AMT_REQ_CREDIT_BUREAU_YEAR": "NBR_DEMANDES_CREDIT_BUREAU_ANNEE",
}

In [ ]:
df.rename(columns=columns, inplace=True)

In [ ]:
df

In [ ]:
# Calculer les taux de NaN pour chaque colonne
nan_counts = df.isna().sum() / len(df) * 100

# Créer une dataframe triée par ordre décroissant des taux de NaN
sorted_nan_counts = nan_counts.sort_values(ascending=False).reset_index()
sorted_nan_counts.columns = ["Colonnes", "Taux_NaN"]


# Créer une coloration (cmap) pour la dataframe
cmap = plt.get_cmap("Greens")
sorted_nan_counts.style.background_gradient(cmap="Oranges", subset=["Taux_NaN"])

In [ ]:
# Calculer les taux de NaN pour chaque colonne
nan_percentages = df.isna().mean() * 100

# Identifier les colonnes avec plus de 60% de NaN
cols_to_drop = nan_percentages[nan_percentages > 30].index

# Supprimer les colonnes identifiées
df = df.drop(cols_to_drop, axis=1)

In [ ]:
df

In [ ]:
df.columns.tolist()

In [ ]:
col = [
    "ID_CLIENT",
    "CIBLE",
    "TYPE_CONTRAT",
    "GENRE",
    "PROPRIETAIRE_VOITURE",
    "PROPRIETAIRE_BIEN_IMMO",
    "NBR_ENFANTS",
    "REVENU_TOTAL",
    "TOTAL_CREDIT",
    "REMB_MENSUEL",
    "MONTANT_PRIX_BIENS",
    "NOM_TYPE_SUITE",
    "TYPE_REVENUS",
    "NIVEAU_EDUCATION",
    "STATUT_FAMILIAL",
    "TYPE_LOGEMENT",
    "POPULATION_REGION",
    "JOURS_NAISSANCE",
    "JOURS_EMPLOI",
    "JOURS_ENREGISTREMENT",
    "JOURS_ID_PUBLICATION",
    "FLAG_MOBILE",
    #'FLAG_TELEPHONE_EMPLOYE',
    #'FLAG_TELEPHONE_TRAVAIL',
    "FLAG_TELEPHONE_MOBILE",
    #'FLAG_TELEPHONE',
    "FLAG_EMAIL",
    "CNT_FAM_MEMBERS",
    "REGION_RATING_CLIENT",
    "REGION_RATING_CLIENT_W_CITY",
    "JOUR_PROCESSUS_DEBUT",
    "HEURE_PROCESSUS_DEBUT",
    "REG_NON_RESIDENCE_REGION",
    "REG_NON_REGION_TRAVAIL",
    "RESIDENCE_NON_TRAVAIL_REGION",
    "REG_NON_RESIDENCE_VILLE",
    "REG_NON_TRAVAIL_VILLE",
    "RESIDENCE_NON_TRAVAIL_VILLE",
    "TYPE_ORGANISATION",
    "SOURCE_EXTERNE_2",
    "SOURCE_EXTERNE_3",
    #'OBS_30_CNT_CERCLE_SOCIAL',
    #'DEF_30_CNT_CERCLE_SOCIAL',
    #'OBS_60_CNT_CERCLE_SOCIAL',
    #'DEF_60_CNT_CERCLE_SOCIAL',
    #'JOURS_DERNIER_CHANGEMENT_TELEPHONE',
    "FLAG_DOCUMENT_2",
    "FLAG_DOCUMENT_3",
    "FLAG_DOCUMENT_4",
    # 'FLAG_DOCUMENT_5',
    # 'FLAG_DOCUMENT_6',
    # 'FLAG_DOCUMENT_7',
    # 'FLAG_DOCUMENT_8',
    # 'FLAG_DOCUMENT_9',
    # 'FLAG_DOCUMENT_10',
    # 'FLAG_DOCUMENT_11',
    # 'FLAG_DOCUMENT_12',
    # 'FLAG_DOCUMENT_13',
    # 'FLAG_DOCUMENT_14',
    # 'FLAG_DOCUMENT_15',
    # 'FLAG_DOCUMENT_16',
    # 'FLAG_DOCUMENT_17',
    # 'FLAG_DOCUMENT_18',
    # 'FLAG_DOCUMENT_19',
    # 'FLAG_DOCUMENT_20',
    # 'FLAG_DOCUMENT_21',
    #'NBR_DEMANDES_CREDIT_BUREAU_HEURE',
    #'NBR_DEMANDES_CREDIT_BUREAU_JOUR',
    #'NBR_DEMANDES_CREDIT_BUREAU_SEMAINE',
    #'NBR_DEMANDES_CREDIT_BUREAU_MOIS',
    #'NBR_DEMANDES_CREDIT_BUREAU_TRIMESTRE',
    "NBR_DEMANDES_CREDIT_BUREAU_ANNEE",
]

In [ ]:
df = df.loc[:, col]

In [ ]:
df.shape

#### **Création colonnes**

In [ ]:
df.columns

**Date de naissance**

In [ ]:
df.JOURS_NAISSANCE

In [ ]:
# Convertir en dates de naissance
df["DATE_NAISSANCE"] = pd.to_datetime("today") + pd.to_timedelta(
    df["JOURS_NAISSANCE"], unit="D"
)


# Supprimer l'heure de la date
df["DATE_NAISSANCE"] = df["DATE_NAISSANCE"].dt.floor("D")

# Changer le format de la date en "jour/mois/année" (format français)
df["DATE_NAISSANCE"] = df["DATE_NAISSANCE"].dt.strftime("%d/%m/%Y")

In [ ]:
df["DATE_NAISSANCE"]

**Age des clients**

In [ ]:
# Convertir en format de date
df["DATE_NAISSANCE"] = pd.to_datetime(df["DATE_NAISSANCE"], format="%d/%m/%Y")

# Calculer l'âge
df["AGE"] = pd.to_datetime("today").year - df["DATE_NAISSANCE"].dt.year

In [ ]:
df.AGE

In [ ]:
df

**Jours travaillés**

In [ ]:
df.JOURS_EMPLOI

In [ ]:
date_list = ["DAYS_BIRTH", "DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH"]
for col in date_list:
    if col in df.columns:
        df[col] = df[col].replace(365243, np.nan)

In [ ]:
df

In [ ]:
# df.drop(columns=['JOURS_NAISSANCE','JOURS_EMPLOI' ], inplace=True)

In [ ]:
df

In [ ]:
df.columns

In [ ]:
# Calculer les taux de NaN pour chaque colonne
nan_counts = df.isna().sum() / len(df) * 100

# Créer une dataframe triée par ordre décroissant des taux de NaN
sorted_nan_counts = nan_counts.sort_values(ascending=False).reset_index()
sorted_nan_counts.columns = ["Colonnes", "Taux_NaN"]


# Créer une coloration (cmap) pour la dataframe
cmap = plt.get_cmap("Greens")
sorted_nan_counts.style.background_gradient(cmap="Blues", subset=["Taux_NaN"])

## **Colonnes finales**

In [ ]:
col_list = [
    "NBR_DEMANDES_CREDIT_BUREAU_ANNEE",
    "NOM_TYPE_SUITE",
    "MONTANT_PRIX_BIENS",
    "CNT_FAM_MEMBERS",
    "PROPRIETAIRE_BIEN_IMMO",
    "JOURS_ENREGISTREMENT",
    "REVENU_TOTAL",
    "TOTAL_CREDIT",
    "REMB_MENSUEL",
    "NIVEAU_EDUCATION",
    "TYPE_REVENUS",
    "FLAG_MOBILE",
    "JOURS_ID_PUBLICATION",
    "NBR_ENFANTS",
    "TYPE_CONTRAT",
    "FLAG_DOCUMENT_4",
    "RESIDENCE_NON_TRAVAIL_VILLE",
    "REG_NON_TRAVAIL_VILLE",
    "DATE_NAISSANCE",
    "AGE",
    "ID_CLIENT",
    "GENRE",
    "CIBLE",
    "REGION_RATING_CLIENT",
    "DATE_DEBUT_TRAVAIL",
    "JOURS_EMPLOI",
    "PROPRIETAIRE_VOITURE",
    "STATUT_FAMILIAL",
    "TYPE_LOGEMENT",
    "SOURCE_EXTERNE_2",
    "SOURCE_EXTERNE_3",
]

In [ ]:
col_fin = [
    "TYPE_CONTRAT",
    "AGE",
    "DATE_NAISSANCE",
    "ID_CLIENT",
    "CIBLE",
    "STATUT_FAMILIAL",
    "JOURS_EMPLOI",
    "TOTAL_CREDIT",
    "REMB_MENSUEL",
    "PROPRIETAIRE_BIEN_IMMO",
    "CNT_FAM_MEMBERS",
    "TYPE_REVENUS",
    "GENRE",
    "MONTANT_PRIX_BIENS",
    "NIVEAU_EDUCATION",
    "REVENU_TOTAL",
    "SOURCE_EXTERNE_2",
    "SOURCE_EXTERNE_3",
    "PROPRIETAIRE_VOITURE",
]

In [ ]:
df.columns.tolist()

In [ ]:
len(col_fin)

In [ ]:
df = df.loc[:, col_fin]

In [ ]:
df

# **Export data**

In [ ]:
df.to_csv("app_train_sample_clean.csv", index=False)

In [ ]:
19/30_000